In [ ]:
import pandas as pd
dtypes  = {"tconst": "string[pyarrow]", "ordering": "int64", "nconst": "string", "category": "string", "job": "string", "characters": "string"}
principals_df = pd.read_csv('title.principals.tsv', sep='\t', dtype=dtypes)
principals_df = principals_df[(principals_df['category'] == 'actor') | (principals_df['category'] == 'actress')]

In [ ]:
principals_df = principals_df[['tconst', 'nconst']].reset_index()
principals_df = principals_df[['tconst', 'nconst']]
amitabh_nconst = 'nm0000821'

In [ ]:
df2 = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz' , compression='gzip', sep='\t')

In [ ]:
indian_movies = df2[(df2['region'] == 'IN') & (df2['language'] == 'hi')]

In [ ]:
df3 = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz' , compression='gzip', sep='\t')

In [ ]:
merged = indian_movies.merge(left_on="titleId", right_on="tconst", right=df3)

In [ ]:
merged = merged[merged['titleType'] == 'movie']
merged = merged[merged['startYear'] != '\\N']
merged = merged.astype({'startYear': 'int32'})
merged = merged[merged['startYear'] > 1968]
merged = merged[merged['isAdult'] == 0]

In [ ]:
# principals_df['mapped_nconst'] = principals_df['nconst'].map(mapping)
grouped = principals_df[['tconst', 'nconst']]


In [ ]:
wewant = merged['tconst']
grouped = grouped.loc[grouped['tconst'].isin(wewant)]
grouped = grouped[grouped['nconst'].map(grouped['nconst'].value_counts()) >= 2]


In [ ]:
grouped

In [ ]:
grouped.reset_index(inplace=True)

In [ ]:
grouped = grouped[['tconst', 'nconst']]


In [ ]:
import numpy as np
def fast_grouper(df):
    keys, values = df.sort_values('tconst').values.T
    uniques, idx = np.unique(keys, True)
    arrays = np.split(values, idx[1:])
    df2 = pd.DataFrame({'tconst':uniques, 'nconst':[list(a) for a in arrays]})
    return df2

In [ ]:
grouped = fast_grouper(grouped)

In [ ]:
grouped.to_csv("grouped_movies_to_actors.csv", index=False)